In [1]:
import cv2
import numpy as np
import dlib
from collections import OrderedDict 
from imutils import face_utils
import imutils
import tensorflow as tf

C:\Users\samrat.sengupta\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\samrat.sengupta\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\samrat.sengupta\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\samrat.sengup

In [2]:
import face_recognition

In [3]:
import deepgaze
from deepgaze.deepgaze.head_pose_estimation import CnnHeadPoseEstimator

[DEEPGAZE] head_pose_estimation.py: the dlib library is installed.


In [4]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [5]:
sess = tf.Session()
head_pose_estimator = CnnHeadPoseEstimator(sess)
head_pose_estimator.load_pitch_variables('./deepgaze/etc/tensorflow/head_pose/pitch/cnn_cccdd_30k.tf')
head_pose_estimator.load_yaw_variables('./deepgaze/etc/tensorflow/head_pose/yaw/cnn_cccdd_30k.tf')
head_pose_estimator.load_roll_variables('./deepgaze/etc/tensorflow/head_pose/roll/cnn_cccdd_30k.tf')






Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./deepgaze/etc/tensorflow/head_pose/pitch/cnn_cccdd_30k.tf
INFO:tensorflow:Restoring parameters from ./deepgaze/etc/tensorflow/head_pose/yaw/cnn_cccdd_30k.tf
INFO:tensorflow:Restoring parameters from ./deepgaze/etc/tensorflow/head_pose/roll/cnn_cccdd_30k.tf


In [6]:
# Left eye points: (36, 37, 38, 39, 40, 41)
# Right eye points: (42, 43, 44, 45, 46, 47)

In [7]:
def midpoint(p1 ,p2):
    return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)

In [8]:
# def yaw2rotmat(yaw):
#     x = 0.0
#     y = 0.0
#     z = yaw
#     ch = np.cos(z)
#     sh = np.sin(z)
#     ca = np.cos(y)
#     sa = np.sin(y)
#     cb = np.cos(x)
#     sb = np.sin(x)
#     rot = np.zeros((3,3), 'float32')
#     rot[0][0] = ch * ca
#     rot[0][1] = sh*sb - ch*sa*cb
#     rot[0][2] = ch*sa*sb + sh*cb
#     rot[1][0] = sa
#     rot[1][1] = ca * cb
#     rot[1][2] = -ca * sb
#     rot[2][0] = -sh * ca
#     rot[2][1] = sh*sa*cb + ch*sb
#     rot[2][2] = -sh*sa*sb + ch*cb
#     return rot

In [ ]:
cap = cv2.VideoCapture(0)
while(True):
    ret, frame = cap.read()
    
# detect faces in the grayscale image
    
    frame = cv2.resize(frame, (560, 560))   
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #frame_number += 1

    # Quit when the input video file ends
    if not ret:
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    #rgb_frame = frame[:, :, ::-1]
    rects = detector(gray, 1)  
    for (i, rect) in enumerate(rects):
    # determine the facial landmarks for the face region, then
    # convert the facial landmark (x, y)-coordinates to a NumPy
    # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)], then draw the face bounding box
        (x, y, w, h) = face_utils.rect_to_bb(rect)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        #show the face number
        cv2.putText(frame, "Face #{}".format(i + 1), (x - 10, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        #loop over the (x, y)-coordinates for the facial landmarks
        #and draw them on the image
        for (x, y) in shape:
            cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)

        landmarks = predictor(gray, rect)
        ly_left_point = (landmarks.part(36).x, landmarks.part(36).y)
        ly_right_point = (landmarks.part(39).x, landmarks.part(39).y)
        
        ly_center_top = midpoint(landmarks.part(37), landmarks.part(38))
        ly_center_bottom = midpoint(landmarks.part(41), landmarks.part(40))
        ly_hor_line = cv2.line(frame, ly_left_point, ly_right_point, (0, 255, 0), 2)
        ly_ver_line = cv2.line(frame, ly_center_top, ly_center_top, (0, 255, 0), 2)
        
        
        ry_left_point = (landmarks.part(42).x, landmarks.part(42).y)
        ry_right_point = (landmarks.part(45).x, landmarks.part(45).y)
    
        ry_center_top = midpoint(landmarks.part(43), landmarks.part(44))
        ry_center_bottom = midpoint(landmarks.part(46), landmarks.part(47))
        ry_hor_line = cv2.line(frame, ry_left_point, ry_right_point, (0, 255, 0), 2)
        ry_ver_line = cv2.line(frame, ry_center_top, ry_center_bottom, (0, 255, 0), 2)    
    

#     # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    face_names = []
    #for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
#         match = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.50)

#         name = None
#         if match[0]:
#             name = "Samrat"
    name = 'Samrat'
    face_names.append(name)

#     # Label the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        if not name:
            continue

        # Draw a box around the face
        cv2.rectangle(frame, (left-60, top-120), (right+60, bottom+50), (0, 0, 255), 2)
        
        image = frame[left-60:right+60,top-120:bottom+50]
        
        image = cv2.resize(image, (480, 480))  

        # Draw a label with a name below the face
        #cv2.rectangle(frame, (left, bottom), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        #cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

        pitch = head_pose_estimator.return_pitch(image,radians=True)
        yaw = head_pose_estimator.return_yaw(image,radians=True)
        roll = head_pose_estimator.return_roll(image,radians=True)
        
        
#         cam_w = frame.shape[1]
#         cam_h = frame.shape[0]
#         c_x = cam_w / 2
#         c_y = cam_h / 2
#         f_x = c_x / np.tan(60/2 * np.pi / 180)
#         f_y = f_x
        
#         camera_distortion = np.float32([0.0, 0.0, 0.0, 0.0, 0.0])

#         camera_matrix = np.float32([[f_x, 0.0, c_x], [0.0, f_y, c_y], [0.0, 0.0, 1.0]])
#         axis = np.float32([[0.0, 0.0, 0.0],[0.0, 0.0, 0.0], [0.0, 0.0, 0.5]])
#         tvec = np.array([0.0, 0.0, 1.0], np.float) # translation vector
        
#         rot_matrix = yaw2rotmat(-yaw[0,0,0])
#         rvec, jacobian = cv2.Rodrigues(rot_matrix)
#         imgpts, jac = cv2.projectPoints(axis, rvec, tvec, camera_matrix, camera_distortion)
        
#         p_start = (int(c_x), int(c_y))
#         p_stop = (int(imgpts[2][0][0]), int(imgpts[2][0][1]))
        
#         cv2.line(frame, p_start, p_stop, (0,0,255), 3) #Red
#         cv2.circle(frame, p_start, 1, (0,255,0), 3) #Green
        
        #print("Estimated camera matrix: \n" + str(camera_matrix) + "\n")
    
        print('data points ','pitch : ' , pitch ,' roll ' ,roll ,' yaw ',yaw)
    
    cv2.imshow('pose', frame)    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

data points  pitch :  [[[0.03217963]]]  roll  [[[-0.00017241]]]  yaw  [[[-0.19646515]]]
data points  pitch :  [[[0.13273744]]]  roll  [[[0.11418705]]]  yaw  [[[-0.42283544]]]
data points  pitch :  [[[0.14273395]]]  roll  [[[0.10289573]]]  yaw  [[[-0.43410265]]]
data points  pitch :  [[[0.12930559]]]  roll  [[[0.11245632]]]  yaw  [[[-0.40388376]]]
data points  pitch :  [[[-0.08411925]]]  roll  [[[-0.05071685]]]  yaw  [[[0.39250508]]]
data points  pitch :  [[[0.04117327]]]  roll  [[[-0.01877896]]]  yaw  [[[0.38506985]]]
data points  pitch :  [[[0.0626554]]]  roll  [[[-0.00962131]]]  yaw  [[[0.398794]]]
data points  pitch :  [[[0.05368046]]]  roll  [[[-0.01459606]]]  yaw  [[[0.38941616]]]
data points  pitch :  [[[0.04697675]]]  roll  [[[-0.02001482]]]  yaw  [[[0.36717257]]]
data points  pitch :  [[[0.05324829]]]  roll  [[[0.11855651]]]  yaw  [[[-0.14684114]]]
data points  pitch :  [[[0.08256634]]]  roll  [[[0.10951939]]]  yaw  [[[-0.250717]]]
data points  pitch :  [[[0.04861128]]]  roll  

In [ ]:
cap.release()
cv2.destroyAllWindows()   